In [1]:
import _init_paths
import numpy as np
import matplotlib.pyplot as plt
import nengo, nengo_loihi

from utils.results_analysis_utils.analyse_results import get_loihi_probes_output
from utils.base_utils.exp_utils import get_grouped_slices_2d_pooling_cl, get_grouped_slices_2d_pooling_cf

# ######################################################

In [2]:
rows, cols, chnls = 3, 3, 3
def get_values(is_cf=False):
  np.random.seed(0)
  if is_cf:
    return np.random.random((chnls, rows, cols))
  else:
    return np.random.random((rows, cols, chnls))

In [3]:
def get_max(grouped_flattened_arr):
  ret = []
  def _neurocore_max_over_one_channel(channel_arr):
    nc_ret = []
    for j in range(0, len(channel_arr), 4):
      nc_ret.append(max(channel_arr[j : j+4]))
    return nc_ret
  
  for i in range(0, len(grouped_flattened_arr), rows*cols):
    ret.extend(_neurocore_max_over_one_channel(grouped_flattened_arr[i : i+(rows*cols)]))
  return ret

## CHANNELS FIRST

In [90]:
vals = get_values(is_cf=True)
print(vals.shape)
print(vals[0, :, :])
print("*"*20)
print(vals[1, :, :])
print("*"*20)
print(vals[2, :, :])
print("*"*20)

(3, 3, 3)
[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]
 [0.43758721 0.891773   0.96366276]]
********************
[[0.38344152 0.79172504 0.52889492]
 [0.56804456 0.92559664 0.07103606]
 [0.0871293  0.0202184  0.83261985]]
********************
[[0.77815675 0.87001215 0.97861834]
 [0.79915856 0.46147936 0.78052918]
 [0.11827443 0.63992102 0.14335329]]
********************


In [91]:
grouped_slices = get_grouped_slices_2d_pooling_cf(pool_size=(2, 2), num_chnls=chnls, rows=rows, cols=cols)
if rows%2 and cols%2:
  rows, cols = rows - 1, cols - 1
num_neurons = chnls * rows * cols
print(vals.flatten()[grouped_slices[:num_neurons]])

[0.5488135  0.71518937 0.54488318 0.4236548  0.38344152 0.79172504
 0.56804456 0.92559664 0.77815675 0.87001215 0.79915856 0.46147936]


In [92]:
ret = get_max(vals.flatten()[grouped_slices[:num_neurons]])
print(ret)
print("*"*20)
ret_reshaped = np.array(ret).reshape(3, rows//2, cols//2)
print(ret_reshaped[0, :, :])
print("*"*20)
print(ret_reshaped[1, :, :])
print("*"*20)
print(ret_reshaped[2, :, :])
print("*"*20)

[0.7151893663724195, 0.925596638292661, 0.8700121482468192]
********************
[[0.71518937]]
********************
[[0.92559664]]
********************
[[0.87001215]]
********************


## ################## CHANNELS LAST

In [7]:
rows, cols, chnls = 3, 3, 3
vals = get_values(is_cf=False)
print(vals.shape)
print(vals[:, :, 0])
print("*"*20)
print(vals[:, :, 1])
print("*"*20)
print(vals[:, :, 2])
print("*"*20)

(3, 3, 3)
[[0.5488135  0.54488318 0.43758721]
 [0.38344152 0.56804456 0.0871293 ]
 [0.77815675 0.79915856 0.11827443]]
********************
[[0.71518937 0.4236548  0.891773  ]
 [0.79172504 0.92559664 0.0202184 ]
 [0.87001215 0.46147936 0.63992102]]
********************
[[0.60276338 0.64589411 0.96366276]
 [0.52889492 0.07103606 0.83261985]
 [0.97861834 0.78052918 0.14335329]]
********************


In [8]:
grouped_slices_cl = get_grouped_slices_2d_pooling_cl(pool_size=(2, 2), num_chnls=chnls, rows=rows, cols=cols)
if rows%2 and cols%2:
  rows, cols = rows - 1, cols - 1
num_neurons = chnls * rows * cols
print(vals.flatten()[grouped_slices_cl[:num_neurons]])

[0.5488135  0.54488318 0.38344152 0.56804456 0.71518937 0.4236548
 0.79172504 0.92559664 0.60276338 0.64589411 0.52889492 0.07103606]


In [9]:
ret = get_max(vals.flatten()[grouped_slices_cl[:num_neurons]])
print(ret)
print("*"*20)
ret_reshaped = np.array(ret).reshape(rows//2, cols//2, 3)
print(ret_reshaped[:, :, 0])
print("*"*20)
print(ret_reshaped[:, :, 1])
print("*"*20)
print(ret_reshaped[:, :, 2])
print("*"*20)

[0.5680445610939323, 0.925596638292661, 0.6458941130666561]
********************
[[0.56804456]]
********************
[[0.92559664]]
********************
[[0.64589411]]
********************


### ##################################################################

In [267]:
def get_reordered(N=rows*cols*chnls//4):
  k = np.arange(N).reshape(3, 2, 2)
  m = np.moveaxis(k, 0, -1)
  return m.flatten()
arr1 = np.arange(12)
print(arr1)
#slices_cl = get_grouped_slices_2d_pooling_cl(pool_size=(1, 1), num_chnls=3, rows=2, cols=2)
# reordered = get_reordered()
# arr1 = arr1[reordered]

reshaped = arr1.reshape(2, 2, 3)
print(reshaped[:, :, 0])
print(reshaped[:, :, 1])
print(reshaped[:, :, 2])

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[[0 3]
 [6 9]]
[[ 1  4]
 [ 7 10]]
[[ 2  5]
 [ 8 11]]


In [264]:
k = np.array([i for i in range(48) if i%4==3]).reshape(3, 2, 2)
m = np.moveaxis(k, 0, -1)
print(m.flatten().reshape(2, 2, 3)[:, :, 0])

# print(np.array([i for i in range(48) if i%4==3]))
# np.array([i for i in range(48) if i%4==3]).reshape(2, 2, 3)[:, :, 0]

[[ 3  7]
 [11 15]]


# #############################################################

In [39]:
grouped_slices = get_grouped_slices_2d_pooling_cf(pool_size=(2, 2), num_chnls=3, rows=3, cols=3)
grouped_slices

array([ 0,  1,  3,  4,  9, 10, 12, 13, 18, 19, 21, 22,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [40]:
grouped_slices = get_grouped_slices_2d_pooling_cl(pool_size=(2, 2), num_chnls=3, rows=3, cols=3)
grouped_slices

array([ 0,  3,  9, 12,  1,  4, 10, 13,  2,  5, 11, 14,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0])